In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
def pointsAboveReplacement(file):
    with open(file, "r") as infile:
        data = json.load(infile)
    for i, player in enumerate(data):
        if i < (len(data)-1):
            data[i].update({"points_above_replacement": player["togga_score"] - data[i+1]["togga_score"]})   
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        pointsAboveReplacement(file)



In [172]:
with open(file, "r") as infile:
        data = json.load(infile)

points_above = np.array([(x["name"], x["points_above_replacement"]) for x in data[:-1]])
#print(points_above)


In [95]:
# Removed ontarget_scoring_att
relevant_forward_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                         'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 
                         'goals_scored', 'key_passes', 'accurateCrosses_total', 'aerialWon_total', 'assists', 
                         'points_above_replacement', 'name', 'age', 'ontarget_scoring_att', 'next_year']

relevant_midfield_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                          'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 'ontarget_scoring_att',
                          'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 'aerialWon_total', 
                          'assists', 'points_above_replacement', 'name', 'age', 'next_year']

relevant_defense_keys = ['goals_conceded', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total',
                         'red_cards', 'minutes', 'successful_dribbles', 'own_goals', 'name', 'ontarget_scoring_att',
                         'interception_total', 'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 
                         'aerialWon_total', 'assists', 'points_above_replacement', 'clearance_total', 'age', 'next_year']

relevant_goalie_keys = ['goals_conceded', 'saves', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'red_cards', 
                        'minutes', 'own_goals', 'interception_total', 'key_passes', 'clean_sheets', 'aerialWon_total', 
                        'penalties_saved', 'points_above_replacement', 'name', 'clearance_total', 'age', 'next_year']

def removeExtraStats(file, relevant_keys):
    with open(file, "r") as infile:
        data = json.load(infile)
    new_data = []
    for i in range(len(data)):
        values = [{key: data[i].get(key)} for key in relevant_keys]
        item = dict(pair for d in values for pair in d.items())
        new_data.append(item)
    with open(file, 'w') as json_data:
        json.dump(new_data, json_data)
        
for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        if position == 'goalies':
            keys = relevant_goalie_keys
        elif position == 'defense':
            keys = relevant_defense_keys
        elif position == 'midfield':
            keys = relevant_midfield_keys
        else:
            keys = relevant_forward_keys
        removeExtraStats(file, keys)

import random

def sogRandom(file, maxVal):
    with open(file, "r") as infile:
        data = json.load(infile)
    for player in data:
        if not player.get("ontarget_scoring_att"):
            player["ontarget_scoring_att"] = random.randint(0,maxVal)
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
    file = 'Data/Sorted_Positions/sorted_defense_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,2)
    file = 'Data/Sorted_Positions/sorted_midfield_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,5)
    file = 'Data/Sorted_Positions/sorted_forwards_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,7)

In [39]:
with open('Data/Sorted_Positions/sorted_forwards_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#print(df)
#df.shape has 38 entries

with open('Data/Sorted_Positions/sorted_forwards_2015+16.json', "r") as infile:
    data = json.load(infile)
df2 = pd.DataFrame(data)
df2 = df2.dropna(axis=1,how='all')
df2 = df2.dropna(axis=0,how='any')

df = df.set_index("name")
df.index.name = None
df2 = df2.set_index("name")
df2.index.name = None

df3 = df2.add(df)
df3 = df3.dropna(axis=1,how='all')
df3 = df3.dropna(axis=0,how='any')

df3.sort_values('togga_score', 0, False)

#df3.shape
#df3 has 17 entries

# Average age of forwards vs. fantasy points
#age_average = df[['age','togga_score']].groupby('age', as_index=False).mean()
#age_average.plot(x='age', y='togga_score', figsize=(15,10), grid=True)


In [38]:
#print(pd.DataFrame(df, columns = list(['name', 'togga_score', 'points_above_replacement'])))
#print(df[["togga_score", "points_above_replacement"]])
#df
#df[["togga_score", "points_above_replacement"]].describe()
#df[["togga_score", "points_above_replacement"]].corr
#df.any()
#%matplotlib inline

#df.hist(column='togga_score', figsize=(15,10))
#df.boxplot(column='togga_score', figsize=(15,20))

#is_highly_rated = df['togga_score'] >= 300.0
#df[is_highly_rated]


In [211]:
with open('Data/Sorted_Positions/sorted_midfield_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 129 entries

In [213]:
#df

In [215]:
with open('Data/Sorted_Positions/sorted_defense_2016+17.json', "r") as infile:
    data = json.load(infile)
    
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 86 entries

In [217]:
#df

## Machine Learning

Load all the one year data for forwards

In [115]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [153]:
with open("Data/positional.json", "r") as infile:
        data = json.load(infile)
        
forwards = []
for player in data['4']:
    forwards.append(pd.DataFrame([player['history']['2015/16']]))
    forwards.append(pd.DataFrame([player['history']['2014/15']]))
    forwards.append(pd.DataFrame([player['history']['2013/14']]))
    forwards.append(pd.DataFrame([player['history']['2012/13']]))

forwards = pd.concat(forwards)
filter = forwards['togga_score'] > 0
forwards = forwards[filter]
keys = relevant_forward_keys.copy()
keys.remove('points_above_replacement')
forwards = forwards[keys]
forwards = forwards.dropna(axis=1,how='all')
forwards = forwards.dropna(axis=0,how='any')

forwards = forwards.set_index("name")
forwards.index.name = None

#forwards.sort_values('togga_score', 0, False).head()



#for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:


# Linear Regressor

In [194]:
x_keys = keys.copy()
x_keys.remove('next_year')
x_keys.remove('name')
X = forwards[x_keys]
Y = forwards['next_year']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=324)
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
Y_prediction = regressor.predict(X_test)
#X_test, y_prediction

RMSE = sqrt(mean_squared_error(y_true = Y_test, y_pred = Y_prediction))
print(RMSE)




100.95872239373855


# Decision Tree Regressor

In [190]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, Y_train)
y_prediction = regressor.predict(X_test)
#y_prediction
#X_train.head()

RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
print(RMSE)

223.37586659659047


In [137]:
import sys
import scipy
import numpy
import matplotlib
import sklearn

In [140]:
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [148]:
# box and whisker plots of some of the stats
plot_keys = ['togga_score', 'goals_scored', 'assists', 'ontarget_scoring_att', 'aerialWon_total', 'minutes', 
             'successful_dribbles', 'age', 'next_year']
forwards[plot_keys].plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
plt.show()

In [147]:
# histograms
forwards[plot_keys].hist()
plt.show()

In [192]:
#array = forwards.values
#print(array[0])
#print(forwards.head(3))
#X = np.asarray(forwards[x_keys])
#Y = np.asarray(forwards['next_year'], dtype=float)
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=324)
#Y_train = list(Y_train.values)
#Y_test = list(Y_test.values)
#X_train = list(X_train.values)
#X_test = list(X_test.values)

seed = 7
scoring = 'accuracy'

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    print(name)
    #kfold = cross_validation.KFold(len(X_train), n_folds=10, random_state=seed)
    #cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    #results.append(cv_results)
    #names.append(name)
    #msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    #print(msg)
    model.fit(X_train, Y_train)
    y_prediction = regressor.predict(X_test)
    RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
    print(RMSE)
    
    
   

LR


ValueError: Unknown label type: array([  69. ,  285.5,  130.5,  558. ,  329. ,    0. ,  345.5,  118. ,
        244.5,  551. ,  427. ,  115. ,   82. ,    6. ,   32. ,  272.5,
          9. ,  555. ,   94. ,    0. ,    0. ,  174. ,  305.5,   94.5,
        117.5,  410.5,    0. ,  514. ,    0. ,   73. ,  326. ,  414. ,
        152. ,  295. ,   86.5,    0. ,  372.5,    0. ,  137. ,  101. ,
        133. ,   -3. ,    0. ,   15. ,  141.5,   22. ,    9. ,  111. ,
        445.5,  560. ,   15. ,    0. ,  293. ,    0. ,   15. ,  143.5,
        290. ,  239.5,  123. ,  410. ,   15. ,  430.5,   30. ,  341. ,
        433. ,  236. ,    0. ,  141.5,  118. ,  578.5])